In [1]:
import requests
from pathlib import Path
import pandas as pd 

In [10]:
output_dir = Path("%pwd").resolve().parent
output_path = output_dir / "apple_data.csv"

In [3]:
# URL de base
base_url = "https://datasets-server.huggingface.co/rows?dataset=Ammok/apple_stock_price_from_1980-2021&config=default&split=train"

# Preparing loop's parameters
response = requests.get(base_url)
data = response.json()
total_line = data['num_rows_total'] # we could request directly num_rows with the hugg_face params size but with this way we economize the first loading
df = pd.json_normalize(data['rows'])  # initialiser la transformation des data en csv

q = total_line // 100
r = total_line % 100
n_iter = q if r==0 else q + 1

# loading data sequentially
for i in range(1,n_iter) :  # commence à 1 car on aura déjà fait au moins un request pour obtenir le total_line même s'il est nul
    offset = i*100
    api_url = f"{base_url}&offset={offset}&length=100"
    # api_url = f"https://datasets-server.huggingface.co/rows?dataset=Ammok/apple_stock_price_from_1980-2021&config=default&split=train&offset={offset}&length=100"
    response = requests.get(api_url)
    data = response.json()
    df = pd.concat([df,pd.json_normalize(data['rows'])], axis = 0, ignore_index = True)

# sortie
df.head(60)
if df.shape[0] - pd.read_csv(output_path).shape[0] > 0 :
    data_csv = df.to_csv(output_path, index=False)

,row_idx,truncated_cells,row.Date,row.Open,row.High,row.Low,row.Close,row.Adj Close,row.Volume
0,0,[],1980-12-12,0.128348,0.128906,0.128348,0.128348,0.100178,469033600
1,1,[],1980-12-15,0.122210,0.122210,0.121652,0.121652,0.094952,175884800
2,2,[],1980-12-16,0.113281,0.113281,0.112723,0.112723,0.087983,105728000
3,3,[],1980-12-17,0.115513,0.116071,0.115513,0.115513,0.090160,86441600
4,4,[],1980-12-18,0.118862,0.119420,0.118862,0.118862,0.092774,73449600
5,5,[],1980-12-19,0.126116,0.126674,0.126116,0.126116,0.098436,48630400
6,6,[],1980-12-22,0.132254,0.132813,0.132254,0.132254,0.103227,37363200
7,7,[],1980-12-23,0.137835,0.138393,0.137835,0.137835,0.107583,46950400
8,8,[],1980-12-24,0.145089,0.145647,0.145089,0.145089,0.113245,48003200
9,9,[],1980-12-26,0.158482,0.159040,0.158482,0.158482,0.123699,55574400


0

In [ ]:
def load_with_for(api_url, output_path) :
    # Preparing loop's parameters
    response = requests.get(api_url)
    data = response.json()
    total_line = data['num_rows_total'] # we could request directly num_rows with the hugg_face params size but with this way we economize the first loading
    df = pd.json_normalize(data['rows'])  # initialiser la transformation des data en csv
    q = total_line // 100
    r = total_line % 100
    n_iter = q if r==0 else q + 1

    # loading data sequentially
    for i in range(1,n_iter) :  # commence à 1 car on aura déjà fait au moins un request pour obtenir le total_line même s'il est nul
        offset = i*100
        api_url = f'"https://datasets-server.huggingface.co/rows?dataset=Ammok/apple_stock_price_from_1980-2021&config=default&split=train&offset={offset}&length=100"'
        response = requests.get(api_url)
        data = response.json()
        df = pd.concat({df,pd.json_normalize(data['rows'])}, axis = 1, ignore_index = True)

    # sortie
    data_csv = df.to_csv(output_path, index=False)  

### old : Exporting data to json file

In [ ]:
output_dir = Path("%pwd").resolve().parent
output_path = output_dir / "hugging_face_data.json"

In [ ]:
import requests
import json
from pathlib import Path

# URL de l'API Hugging Face
api_url = "https://datasets-server.huggingface.co/rows?dataset=Ammok/apple_stock_price_from_1980-2021&config=default&split=train&offset=0&length=100"

# Exécuter la requête avec requests
response = requests.get(api_url)

# Vérifier si la requête a réussi (code 200)
if response.status_code == 200:
    # Charger les données JSON
    data = response.json()

    # Sauvegarder les données dans un fichier JSON (par exemple)
    output_path = Path(__file__).resolve().parent / "hugging_face_data.json"
    with open(
        output_path, "w"
    ) as f:  # with garantit la fermeture auto du fichier même en cas d'erreur
        json.dump(data, f, indent=2)

    print(f"Données enregistrées avec succès dans {output_path}")
else:
    print(f"Échec de la requête. Code d'erreur : {response.status_code}")